<a href="https://colab.research.google.com/github/nishkalavallabhi/practicalnlp/blob/V_2_0/Ch6/chatbot-code/atis_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
import os
import pickle
import numpy as np
import pandas as pd

In [0]:
def atisfold(fold):
    assert fold in range(5)
    filename = os.path.join('/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data','atis.fold'+str(fold)+'.pkl')
    f = open(filename, 'rb')
    try:
        train_set, valid_set, test_set, dicts = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set, dicts = pickle.load(f)
    return train_set, valid_set, test_set, dicts

In [0]:
train_set, valid_set, test_set, dic = atisfold(0) 

In [6]:
def rev_map(d):
    return dict((d[i],i) for i in d)

labels2idx = rev_map(dic['labels2idx'])
tables2idx = rev_map(dic['tables2idx'])
words2idx = rev_map(dic['words2idx'])

word_indices = train_set[0]
name_entities = train_set[1]
labels = train_set[2]

print ("Number of sentences : ",len(word_indices))

def display(n):
    sense = []
    for i in range(len(word_indices[n])):
    #     sense.append({"word_index":word_indices[0][i],"word":words2idx[word_indices[0][i]],"entity_index":name_entities[0][i],"entity":tables2idx[name_entities[0][i]],"label_index":labels[0][i],"label":labels2idx[labels[0][i]]})
        sense.append({"word":words2idx[word_indices[n][i]],"entity":tables2idx[name_entities[n][i]],"label":labels2idx[labels[n][i]]})
    return pd.DataFrame(sense)

Number of sentences :  3983


In [7]:
display(2)

,entity,label,word
0,<NOTABLE>,O,i
1,<NOTABLE>,O,'d
2,<NOTABLE>,O,like
3,<NOTABLE>,O,to
4,<NOTABLE>,O,know
5,<NOTABLE>,O,the
6,B-flight_mod,B-flight_mod,earliest
7,<NOTABLE>,O,flight
8,<NOTABLE>,O,from
9,B-city_name,B-fromloc.city_name,boston


# Reading IOB files from data2/

In [0]:
def get_data(filename):
    df = pd.read_csv(filename,delim_whitespace=True,names=['word','label'])
    beg_indices = list(df[df['word'] == 'BOS'].index)+[df.shape[0]]
    sents,labels,intents = [],[],[]
    for i in range(len(beg_indices[:-1])):
        sents.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['word'].values)
        labels.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['label'].values)
        intents.append(df.loc[beg_indices[i+1]-1]['label'])    
    return np.array(sents),np.array(labels),np.array(intents)

def get_data2(filename):
    with open(filename) as f:
        contents = f.read()
    sents,labels,intents = [],[],[]
    for line in contents.strip().split('\n'):
        words,labs = [i.split(' ') for i in line.split('\t')]
        sents.append(words[1:-1])
        labels.append(labs[1:-1])
        intents.append(labs[-1])
    return np.array(sents),np.array(labels),np.array(intents)

read_method = {'/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis-2.dev.w-intent.iob':get_data,
               '/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis.train.w-intent.iob':get_data2,
               '/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis.test.w-intent.iob':get_data,
              '/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis-2.train.w-intent.iob':get_data2}

def fetch_data(fname):
    func = read_method[fname]
    return func(fname)

In [24]:
sents,labels,intents = fetch_data('/content/drive/NLP_book/Datasets/practicalnlp-master/Ch6/chatbot-code/data2/atis.train.w-intent.iob')
def display(n,intents):
    sense = []
    print ("INTENT : ",intents[n])
    for i in range(len(sents[n])):
    #     sense.append({"word_index":word_indices[0][i],"word":words2idx[word_indices[0][i]],"entity_index":name_entities[0][i],"entity":tables2idx[name_entities[0][i]],"label_index":labels[0][i],"label":labels2idx[labels[0][i]]})
        sense.append({"word":sents[n][i],"label":labels[n][i]})
    return pd.DataFrame(sense)

display(1)

TypeError: ignored